# Müzik Türü Sınıflandırma Projesi:

Bu notebook, FMA (Free Music Archive) veri setini kullanarak müzik türü sınıflandırma modeli geliştirmek için veri hazırlama ve dengeleme işlemlerini içermektedir.

## Gerekli Kütüphanelerin İçe Aktarılması:

Aşağıdaki hücrede, projede kullanılacak temel Python kütüphaneleri import edilmektedir:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE
from collections import Counter

%matplotlib inline
sns.set(style='whitegrid')

# Yardımcı Fonksiyonlar
## Sınıf Dağılımı Görselleştirme Fonksiyonu
Aşağıdaki fonksiyon, veri setindeki sınıf dağılımlarını görselleştirmek için kullanılacaktır. Bu görselleştirme, veri dengesizliğini anlamamıza yardımcı olur.

In [ ]:
def plot_class_distribution(y, labels, title):
    counts = pd.Series(y).value_counts().sort_index()
    valid_indices = counts.index[counts.index < len(labels)]
    counts = counts.loc[valid_indices]
    names = labels[counts.index]

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x=names, y=counts.values, hue=names, palette='viridis', legend=False)
    ax.set_title(title)
    ax.set_xlabel('Sınıf')
    ax.set_ylabel('Sayı')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Veri Yükleme ve Ön İşleme
Bu bölümdeki fonksiyon:

FMA metadata dosyalarını yükler
Gerekli sütunları seçer
Eksik verileri temizler
Etiketleri kodlar
Veriyi sayısal formata dönüştürür

In [ ]:
def load_data():
    tracks_path = 'fma_metadata/tracks.csv'
    features_path = 'fma_metadata/features.csv'

    if not os.path.exists(tracks_path) or not os.path.exists(features_path):
        raise FileNotFoundError(f"Gerekli veri dosyaları bulunamadı. '{tracks_path}' ve '{features_path}' dosyalarının mevcut olduğundan emin olun.")

    tracks = pd.read_csv(tracks_path, index_col=0, header=[0,1])
    
    features = pd.read_csv(features_path, index_col=0, header=[0,1])  # Çok seviyeli başlıkla oku
    features = features.loc[:, features.columns.get_level_values(0) != 'statistics']  # 'statistics' sütunlarını kaldır
    features = features.astype(np.float32)  # Sayısal olmayan sütunları kaldırdıktan sonra float'a dönüştür

    features.index = features.index.astype(str)
    tracks.index = tracks.index.astype(str)

    genre_series = tracks[('track', 'genre_top')].dropna()
    common_index = features.index.intersection(genre_series.index)

    X = features.loc[common_index]
    y_labels = genre_series.loc[common_index]

    X = X.fillna(0).replace([np.inf, -np.inf], 0).astype(np.float32)

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_labels)

    print('Veriler yüklendi ve önişlendi.')
    return X, y, label_encoder

# Başlangıç Veri Analizi
Verinin ilk yüklemesini yapıp, başlangıçtaki sınıf dağılımını inceleyelim. Bu analiz, veri dengesizliği problemini görselleştirmemize yardımcı olacak.

In [ ]:
# Veriyi yükle ve önişle
X, y, le = load_data()

# Başlangıç dağılımını göster
plot_class_distribution(y, le.classes_, 'Başlangıç Sınıf Dağılımı')

# Veri Bölme ve Eğitim Seti Analizi
Veriyi eğitim ve test setlerine ayırıp, eğitim setindeki sınıf dağılımını inceliyoruz. Stratified split kullanarak orijinal dağılımı koruyoruz.

In [ ]:
# Veriyi böl ve eğitim dağılımını göster
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

plot_class_distribution(y_train, le.classes_, 'Eğitim Seti Dağılımı')
print(f'Eğitim/test bölünmesi tamamlandı: X_train {X_train.shape}, X_test {X_test.shape}')

# Detaylı dağılımı yazdır
unique, counts = np.unique(y_train, return_counts=True)
print("\nEğitim Seti Dağılımı (ham sayılar):")
for i, (u, c) in enumerate(zip(unique, counts)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

# Veri Dengeleme - Aşama 1
İlk aşamada, çok az örneğe sahip sınıflar için RandomOverSampler kullanılıyor. Bu aşama, BorderlineSMOTE için yeterli örnek sayısına ulaşmamızı sağlar.

In [ ]:
# Adım 1: En az temsil edilen sınıflar için RandomOverSampler
print('\nAdım 1: Aşırı az temsil edilen sınıflar için RandomOverSampler uygulanıyor...')
min_samples_threshold = 20  # BorderlineSMOTE için gereken minimum örnek sayısı
ros = RandomOverSampler(sampling_strategy={3: min_samples_threshold}, random_state=42)
X_partial, y_partial = ros.fit_resample(X_train, y_train)

# Ara sonuçları göster
unique_partial, counts_partial = np.unique(y_partial, return_counts=True)
print("\nRandomOverSampler sonrası dağılım (ham sayılar):")
for i, (u, c) in enumerate(zip(unique_partial, counts_partial)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

plot_class_distribution(y_partial, le.classes_, 'RandomOverSampler Sonrası')

# Veri Dengeleme - Aşama 2
İkinci aşamada, daha sofistike bir yaklaşım olan BorderlineSMOTE kullanılarak kalan sınıflar dengeleniyor. Bu yöntem, sadece rastgele kopyalama yerine sentetik örnekler oluşturur.

Not: Bu aşama, veri setinin yapısına bağlı olarak başarısız olabilir. Bu durumda, ilk aşamadaki sonuçlar kullanılacaktır.

In [ ]:
# Adım 2: Kalan sınıflar için BorderlineSMOTE
print('\nAdım 2: Kalan sınıflar için BorderlineSMOTE uygulanıyor...')
borderline_smote = BorderlineSMOTE(random_state=42)

try:
    X_res, y_res = borderline_smote.fit_resample(X_partial, y_partial)
    print(f'Kombine örnekleme tamamlandı: X_res {X_res.shape}, y_res {y_res.shape}')
    
    # Son dağılımı yazdır ve göster
    unique_res, counts_res = np.unique(y_res, return_counts=True)
    print("\nSon Dağılım (ham sayılar):")
    for i, (u, c) in enumerate(zip(unique_res, counts_res)):
        print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")
    
    plot_class_distribution(y_res, le.classes_, 'Son Dengelenmiş Dağılım')

except Exception as e:
    print(f'BorderlineSMOTE örnekleme başarısız oldu: {e} - kısmi örneklenmiş veri kullanılıyor')
    X_res, y_res = X_partial, y_partial
    plot_class_distribution(y_res, le.classes_, 'Kısmi Örnekleme (BorderlineSMOTE başarısız)')

print("\nİşlem hattı tamamlandı. Yeniden örneklenmiş eğitim verisi (X_res, y_res) ve test verisi (X_test, y_test) hazır.")

# Özellik Seçimi (K-Best Feature Selection)
Model performansını artırmak ve aşırı öğrenmeyi (overfitting) azaltmak için K-Best özellik seçimi algoritmasını uygulayacağız. Bu algoritma, her özelliğin hedef değişkenle olan istatistiksel ilişkisini ölçer ve en anlamlı K özelliği seçer.

In [ ]:
# K-Best özellik seçimi uygulaması
print('\nK-Best özellik seçimi uygulanıyor...')

k = 250  # Seçilecek özellik sayısı
print(f"Toplam özellik sayısı: {X_res.shape[1]}, Seçilecek özellik sayısı: {k}")

# SelectKBest ile özellik seçimi
selector = SelectKBest(score_func=f_classif, k=k)
X_res_selected = selector.fit_transform(X_res, y_res)
X_test_selected = selector.transform(X_test)

# Hangi özelliklerin seçildiğini gösteren görselleştirme
selected_mask = selector.get_support()
scores = selector.scores_
feature_indices = np.arange(len(selected_mask))

plt.figure(figsize=(12, 6))
plt.bar(feature_indices, scores, alpha=0.3, color='g')
plt.bar(feature_indices[selected_mask], scores[selected_mask], color='g')
plt.title('Özellik Skorları ve Seçilen Özellikler')
plt.xlabel('Özellik İndeksi')
plt.ylabel('F-değeri (F-value)')
plt.tight_layout()
plt.show()

print(f"Özellik seçimi tamamlandı. Seçilen özelliklerin boyutu: {X_res_selected.shape}")

# Orijinal veriyi güncellenmiş veri ile değiştirelim
X_res = X_res_selected
X_test = X_test_selected

# PyTorch LSTM MODEL EĞİTİMİ


In [ ]:
# Dengelenmiş veri setinden doğrulama seti ayır
X_train_bal, X_val, y_train_bal, y_val = train_test_split(
    X_res, y_res, test_size=0.1, stratify=y_res, random_state=42
)

# Veri Ölçeklendirme (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_bal)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Veri ölçeklendirme tamamlandı.")
print(f"Ölçeklenmiş eğitim verisi boyutu: {X_train_scaled.shape}")
print(f"Ölçeklenmiş doğrulama verisi boyutu: {X_val_scaled.shape}")
print(f"Ölçeklenmiş test verisi boyutu: {X_test_scaled.shape}")

# LSTM Modeli için Veri Hazırlığı
PyTorch LSTM modeli için, veriyi uygun formata dönüştürmemiz gerekir. LSTM modeller sıralı veri bekler, bu nedenle öznitelik vektörünü zamansal bir diziye dönüştüreceğiz.

In [ ]:
# PyTorch tensörlerine dönüştürme ve veri setlerini hazırlama
def create_sequence_data(X, y, sequence_length=10):
    """
    Öznitelik vektörünü sıralı verilere dönüştürür.
    FMA veri seti sıralı yapıda değil, bu nedenle yapay bir sıra oluşturuyoruz.
    """
    # Veri boyutlarını kontrol et
    n_samples, n_features = X.shape
    
    # Veriyi yeniden şekillendirme
    features_per_timestep = n_features // sequence_length
    
    if features_per_timestep == 0:
        features_per_timestep = 1
        sequence_length = min(sequence_length, n_features)
    
    # Son timestep'e sığmayan özellikleri ele alma
    remainder = n_features - (sequence_length * features_per_timestep)
    
    # Yeniden şekillendirilmiş veri için array oluşturma
    X_seq = np.zeros((n_samples, sequence_length, features_per_timestep))
    
    # Veriyi yeniden şekillendirme
    for i in range(n_samples):
        for t in range(sequence_length):
            start_idx = t * features_per_timestep
            end_idx = min(start_idx + features_per_timestep, n_features)
            
            if start_idx < n_features:
                X_seq[i, t, :end_idx-start_idx] = X[i, start_idx:end_idx]
    
    # PyTorch tensörlerine dönüştürme
    X_tensor = torch.FloatTensor(X_seq)
    y_tensor = torch.LongTensor(y)
    
    return X_tensor, y_tensor

# Sıralı veri için hiperparametre
sequence_length = 5

# Ölçeklenmiş verileri sıralı forma dönüştürme
X_train_seq, y_train_tensor = create_sequence_data(X_train_scaled, y_train_bal, sequence_length)
X_val_seq, y_val_tensor = create_sequence_data(X_val_scaled, y_val, sequence_length)
X_test_seq, y_test_tensor = create_sequence_data(X_test_scaled, y_test, sequence_length)

print(f"Eğitim veri boyutu: {X_train_seq.shape}")
print(f"Doğrulama veri boyutu: {X_val_seq.shape}")
print(f"Test veri boyutu: {X_test_seq.shape}")

# PyTorch DataLoader oluşturma
batch_size = 512
train_dataset = TensorDataset(X_train_seq, y_train_tensor)
val_dataset = TensorDataset(X_val_seq, y_val_tensor)
test_dataset = TensorDataset(X_test_seq, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# LSTM Model Tanımı ve Eğitimi
Aşağıda müzik türü sınıflandırması için bir LSTM (Long Short-Term Memory) ağı tanımlıyoruz. LSTM'ler, müzik gibi sıralı verilerde başarılı olan bir derin öğrenme mimarisidir.

In [ ]:
# LSTM model sınıfını tanımlama
class MusicGenreLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(MusicGenreLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM katmanları
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Batch normalization
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        
        # Dropout katmanı
        self.dropout = nn.Dropout(dropout)
        
        # Tam bağlantılı katmanlar
        self.fc1 = nn.Linear(hidden_size, 128)  # İlk tam bağlantılı katman
        self.fc2 = nn.Linear(128, num_classes)
        
        # Aktivasyon fonksiyonları
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # LSTM katmanından geçirme
        # x şekli: (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        
        # Son zaman adımının çıktısını al
        # lstm_out şekli: (batch_size, sequence_length, hidden_size)
        lstm_out = lstm_out[:, -1, :]
        
        # Batch normalization
        batch_norm_out = self.batch_norm(lstm_out)
        
        # İlk tam bağlantılı katman
        fc1_out = self.fc1(batch_norm_out)
        fc1_out = self.relu(fc1_out)
        fc1_out = self.dropout(fc1_out)
        
        # İkinci tam bağlantılı katman (çıkış katmanı)
        out = self.fc2(fc1_out)
        
        return out

# Model parametreleri
input_size = X_train_seq.shape[2]  # Bir zaman adımındaki özellik sayısı
hidden_size = 128  # LSTM gizli katman boyutu
num_layers = 2  # LSTM katman sayısı
num_classes = len(le.classes_)  # Sınıf sayısı
dropout = 0.3

# GPU kullanılabilir mi kontrol et
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

# Model oluşturma
model = MusicGenreLSTM(input_size, hidden_size, num_layers, num_classes, dropout).to(device)
print(model)

# Kayıp fonksiyonu ve optimize edici tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Eğitim fonksiyonu
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50, early_stopping_patience=5, min_improvement_threshold=0.001):
    # Ölçüm değerlerini saklayacak listeler
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    # En iyi doğrulama kaybını ve modeli saklama
    # min_improvement_threshold: Doğrulama kaybındaki minimum iyileşme eşiği, 
    # bunun altındaki iyileşmeler anlamlı kabul edilmez ve erken durdurma sayacı sıfırlanmaz.
    best_val_loss = float('inf')
    best_model = None
    
    # Erken durdurma için sayaç ve sabır parametresi
    early_stopping_counter = 0
    
    for epoch in range(num_epochs):
        # Eğitim modu
        model.train()
        
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Gradyanları sıfırla
            optimizer.zero_grad()
            
            # İleri geçiş
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Geri yayılım ve optimize etme
            loss.backward()
            optimizer.step()
            
            # İstatistikleri güncelle
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Doğrulama modu
        model.eval()
        
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # İleri geçiş
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # İstatistikleri güncelle
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Epoch sonuçlarını hesapla
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_train_acc = train_correct / train_total
        epoch_val_acc = val_correct / val_total
        
        # Öğrenme oranını ayarla
        scheduler.step(epoch_val_loss)
        
        # Sonuçları sakla
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_accs.append(epoch_train_acc)
        val_accs.append(epoch_val_acc)
        
        # Eğitim durumunu yazdır
        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, '
              f'Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}')
        
        # En iyi modeli sakla ve erken durdurma durumunu kontrol et
        # Doğrulama kaybındaki iyileşme miktarını hesapla
        improvement = best_val_loss - epoch_val_loss
        
        if epoch_val_loss < best_val_loss:
            # Eğer iyileşme miktarı eşik değerinden fazlaysa sayacı sıfırla
            if improvement > min_improvement_threshold:
                early_stopping_counter = 0  # Counter sıfırla
                print(f'Validation loss improved by {improvement:.6f}, which is above threshold ({min_improvement_threshold:.6f})')
            else:
                # İyileşme var ama eşik değerinin altında, bu durumda counter'ı artırıyoruz
                early_stopping_counter += 1
                print(f'Validation loss improved by only {improvement:.6f}, which is below threshold ({min_improvement_threshold:.6f})')
            
            # En iyi modeli ve validation loss değerini her durumda güncelle
            best_val_loss = epoch_val_loss
            best_model = model.state_dict()
        else:
            early_stopping_counter += 1  # Counter artır
            
        # Erken durdurma kontrolü
        if early_stopping_counter >= early_stopping_patience:
            print(f'Erken durdurma: Validation loss {early_stopping_patience} epoch boyunca yeterince iyileşmedi (minimum eşik: {min_improvement_threshold:.6f}).')
            break
    
    # En iyi model ağırlıklarını yükle
    model.load_state_dict(best_model)
    
    return model, train_losses, val_losses, train_accs, val_accs

# Modeli eğit
print("Model eğitimi başlıyor...")
num_epochs = 50
early_stopping_patience = 3  # Model belirli bir eşik değerinden fazla iyileşmezse, bu sayıda epoch sonra eğitimi durdur

try:
    # Doğrulama kaybında 0.02 altındaki iyileşmeleri önemsiz olarak kabul et
    min_improvement_threshold = 0.02  
    
    model, train_losses, val_losses, train_accs, val_accs = train_model(
        model, train_loader, val_loader, criterion, optimizer, scheduler, 
        num_epochs=num_epochs, early_stopping_patience=early_stopping_patience,
        min_improvement_threshold=min_improvement_threshold
    )
    print("Model eğitimi tamamlandı!")
except KeyboardInterrupt:
    print("Eğitim kullanıcı tarafından durduruldu.")

# Model Değerlendirmesi ve Görselleştirme
Bu bölümde eğitilmiş modeli test veri seti üzerinde değerlendirip, sonuçları görselleştireceğiz.

In [ ]:
# Eğitim sonuçlarını görselleştirme
def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(14, 5))
    
    # Kayıp grafiği
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Eğitim', marker='o')
    plt.plot(val_losses, label='Doğrulama', marker='*')
    plt.title('Model Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp (Cross-Entropy)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Doğruluk grafiği
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Eğitim', marker='o')
    plt.plot(val_accs, label='Doğrulama', marker='*')
    plt.title('Model Doğruluğu')
    plt.xlabel('Epoch')
    plt.ylabel('Doğruluk')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# Eğitim sonuçlarını görselleştir
try:
    plot_training_history(train_losses, val_losses, train_accs, val_accs)
except NameError:
    print("Eğitim geçmişi bulunamadı. Önce modeli eğitin.")

# Test veri seti üzerinde değerlendirme
def evaluate_model(model, test_loader, device):
    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    # Doğruluk hesapla
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    
    # Sonuçları yazdır
    print(f"Test Doğruluğu: {accuracy:.4f}")
    
    # Sınıflandırma raporu
    print("\nSınıflandırma Raporu:")
    print(classification_report(y_true, y_pred, target_names=le.classes_))
    
    # Karmaşıklık matrisi
    plt.figure(figsize=(12, 10))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title('Karmaşıklık Matrisi')
    plt.xlabel('Tahmin Edilen Etiketler')
    plt.ylabel('Gerçek Etiketler')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return y_true, y_pred

# Test veri seti üzerinde değerlendir
try:
    y_true, y_pred = evaluate_model(model, test_loader, device)
except NameError:
    print("Model bulunamadı. Önce modeli eğitin.")

In [ ]:
# ROC Curve Analysis for Multiclass Classification
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

def plot_multiclass_roc_curve(model, test_loader, device, label_encoder, title="ROC Curves"):
    """Plot ROC curves for multiclass classification"""
    model.eval()
    all_predictions_proba = []
    all_true_labels = []
    
    # Get prediction probabilities
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            # Apply softmax to get probabilities
            proba = torch.softmax(outputs, dim=1)
            all_predictions_proba.extend(proba.cpu().numpy())
            all_true_labels.extend(labels.cpu().numpy())
    
    y_true = np.array(all_true_labels)
    y_proba = np.array(all_predictions_proba)
    n_classes = len(label_encoder.classes_)
    
    # Binarize the labels for multiclass ROC
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_proba.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # Plot ROC curves
    plt.figure(figsize=(12, 8))
    
    # Plot micro-average ROC curve
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'Micro-average ROC (AUC = {roc_auc["micro"]:.3f})',
             color='deeppink', linestyle=':', linewidth=4)
    
    # Plot ROC curve for each class
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'purple', 'brown', 'pink'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, linewidth=2,
                 label=f'{label_encoder.classes_[i]} (AUC = {roc_auc[i]:.3f})')
    
    # Plot random classifier line
    plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title(f'{title} - Multiclass ROC Curves', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print AUC summary
    print(f"\n📊 ROC AUC Scores:")
    print(f"{'Genre':<15} {'AUC Score':<10}")
    print(f"{'='*25}")
    for i, genre in enumerate(label_encoder.classes_):
        print(f"{genre:<15} {roc_auc[i]:<10.3f}")
    print(f"{'='*25}")
    print(f"{'Micro-Average':<15} {roc_auc['micro']:<10.3f}")
    
    return roc_auc

# Plot ROC curves for the LSTM model
print("\n🎭 Generating ROC Curves for Multiclass Classification...")
roc_scores = plot_multiclass_roc_curve(model, test_loader, device, le, 
                                      "LSTM Model")

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, f1_score

def plot_f1_scores_by_genre(y_true, y_pred, class_names, title="F1 Scores by Genre"):
    """Plot F1 scores for each genre with detailed visualization"""
    
    # Calculate F1 scores for each class
    precision, recall, f1_scores, support = precision_recall_fscore_support(y_true, y_pred, average=None)
    
    # Calculate macro and weighted averages
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    
    # Create the plot
    plt.figure(figsize=(14, 8))
    
    # Create bar plot
    x_pos = np.arange(len(class_names))
    bars = plt.bar(x_pos, f1_scores, alpha=0.8, 
                   color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
                         '#8c564b', '#e377c2', '#7f7f7f'][:len(class_names)])
    
    # Customize the plot
    plt.xlabel('Music Genres', fontsize=12, fontweight='bold')
    plt.ylabel('F1 Score', fontsize=12, fontweight='bold')
    plt.title(f'{title}\nMacro Avg: {f1_macro:.3f} | Weighted Avg: {f1_weighted:.3f}', 
              fontsize=14, fontweight='bold')
    plt.xticks(x_pos, class_names, rotation=45, ha='right')
    plt.ylim(0, 1.0)
    
    # Add value labels on bars
    for i, (bar, score, support_count) in enumerate(zip(bars, f1_scores, support)):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{score:.3f}\n(n={support_count})',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Add horizontal lines for averages
    plt.axhline(y=f1_macro, color='red', linestyle='--', alpha=0.7, 
                label=f'Macro Average: {f1_macro:.3f}')
    plt.axhline(y=f1_weighted, color='orange', linestyle='--', alpha=0.7, 
                label=f'Weighted Average: {f1_weighted:.3f}')
    
    # Add legend and grid
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    # Print detailed F1 score analysis
    print(f"\n📊 F1 Score Analysis by Genre:")
    print(f"{'Genre':<15} {'F1 Score':<10} {'Precision':<10} {'Recall':<10} {'Support':<10}")
    print(f"{'='*65}")
    
    for i, genre in enumerate(class_names):
        print(f"{genre:<15} {f1_scores[i]:<10.3f} {precision[i]:<10.3f} {recall[i]:<10.3f} {support[i]:<10}")
    
    print(f"{'='*65}")
    print(f"{'Macro Avg':<15} {f1_macro:<10.3f} {np.mean(precision):<10.3f} {np.mean(recall):<10.3f} {np.sum(support):<10}")
    print(f"{'Weighted Avg':<15} {f1_weighted:<10.3f} {np.average(precision, weights=support):<10.3f} {np.average(recall, weights=support):<10.3f} {np.sum(support):<10}")
    
    # Identify best and worst performing genres
    best_genre_idx = np.argmax(f1_scores)
    worst_genre_idx = np.argmin(f1_scores)
    
    print(f"\n🏆 Best Performing Genre: {class_names[best_genre_idx]} (F1: {f1_scores[best_genre_idx]:.3f})")
    print(f"🔍 Needs Improvement: {class_names[worst_genre_idx]} (F1: {f1_scores[worst_genre_idx]:.3f})")
    
    # Performance categories
    excellent_genres = [class_names[i] for i, score in enumerate(f1_scores) if score >= 0.8]
    good_genres = [class_names[i] for i, score in enumerate(f1_scores) if 0.6 <= score < 0.8]
    poor_genres = [class_names[i] for i, score in enumerate(f1_scores) if score < 0.6]
    
    print(f"\n📈 Performance Categories:")
    if excellent_genres:
        print(f"   🟢 Excellent (≥0.8): {', '.join(excellent_genres)}")
    if good_genres:
        print(f"   🟡 Good (0.6-0.8): {', '.join(good_genres)}")
    if poor_genres:
        print(f"   🔴 Needs Work (<0.6): {', '.join(poor_genres)}")
    
    return f1_scores, f1_macro, f1_weighted

# Plot F1 scores for the LSTM model
print("\n📊 Generating F1 Score Analysis for Each Genre...")
f1_individual, f1_macro_score, f1_weighted_score = plot_f1_scores_by_genre(
    y_true, y_pred, le.classes_, 
    "LSTM Model - F1 Scores by Genre"
)